In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/paligemma-result/paligemma_8.csv
/kaggle/input/paligemma-result/paligemma_baseline.csv


In [4]:
import torch
from transformers import AutoModel, AutoTokenizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

phobert_model = AutoModel.from_pretrained("vinai/phobert-base")
phobert_tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")

def tokenize_and_embed(text, model, tokenizer):
    tokens = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**tokens)
    embeddings = outputs.last_hidden_state.mean(dim=1).numpy()
    return embeddings

def evaluate_wump(input_text, output_text, model, tokenizer):
    input_embedding = tokenize_and_embed(input_text, model, tokenizer)
    output_embedding = tokenize_and_embed(output_text, model, tokenizer)
    
    semantic_similarity = cosine_similarity(input_embedding, output_embedding)[0][0]
    
    return semantic_similarity
input_text = "Con cáo nâu nhanh nhẹn nhảy qua con chó lười biếng."
output_text = "Một con cáo màu nâu nhanh chóng nhảy qua một con chó lười biếng."

wump_scores = evaluate_wump(input_text, output_text, phobert_model, phobert_tokenizer)
print(wump_scores)


config.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/543M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


0.9158007


In [36]:
df = pd.read_csv('/kaggle/input/paligemma-result/paligemma_baseline.csv')
print(df.head())

   Unnamed: 0                 question          answer  img_id  type  \
0           0   màu của miếng vá là gì  màu xanh dương  557067     2   
1           1         màu của áo là gì         màu cam  436394     2   
2           2         màu của áo là gì  màu xanh dương  541050     2   
3           3  màu của căn phòng là gì       màu trắng  426077     2   
4           4   màu của quả bóng là gì          màu đỏ  314710     2   

                                               image          predictions  
0  <PIL.Image.Image image mode=RGB size=500x374 a...  \nMàu xanh da trời.  
1  <PIL.Image.Image image mode=RGB size=640x428 a...            \nquả cam  
2  <PIL.Image.Image image mode=RGB size=480x640 a...  \nMàu xanh da trời.  
3  <PIL.Image.Image image mode=RGB size=480x640 a...              \ntrắng  
4  <PIL.Image.Image image mode=RGB size=640x427 a...            \nMàu đỏ.  


In [37]:
scores = []
for (answer, predict) in zip(df['answer'].tolist(), df['predictions'].tolist()):
    scores.append(evaluate_wump(answer, predict, phobert_model, phobert_tokenizer))
df['score'] = scores

In [38]:
def convert_score(score):
    if score >= 0.8:
        return 1
    return 0
df['score'] = df['score'].apply(convert_score)

In [39]:
sub_df = df[df['score'] == 0]
print(sub_df.head())

   Unnamed: 0                 question          answer  img_id  type  \
0           0   màu của miếng vá là gì  màu xanh dương  557067     2   
1           1         màu của áo là gì         màu cam  436394     2   
2           2         màu của áo là gì  màu xanh dương  541050     2   
3           3  màu của căn phòng là gì       màu trắng  426077     2   
4           4   màu của quả bóng là gì          màu đỏ  314710     2   

                                               image          predictions  \
0  <PIL.Image.Image image mode=RGB size=500x374 a...  \nMàu xanh da trời.   
1  <PIL.Image.Image image mode=RGB size=640x428 a...            \nquả cam   
2  <PIL.Image.Image image mode=RGB size=480x640 a...  \nMàu xanh da trời.   
3  <PIL.Image.Image image mode=RGB size=480x640 a...              \ntrắng   
4  <PIL.Image.Image image mode=RGB size=640x427 a...            \nMàu đỏ.   

   score  
0      0  
1      0  
2      0  
3      0  
4      0  


In [40]:
import plotly.graph_objects as go

type_counts = df['type'].value_counts()
fig = go.Figure(data=[go.Pie(labels=type_counts.index, values=type_counts.values)])
fig.update_layout(title='Distribution of Data Types')
fig.show()

In [42]:
grouped = df.groupby('type')['score'].value_counts().unstack().fillna(0)
grouped.columns = ['Wrong Predictions', 'Correct Predictions']

grouped['Total'] = grouped['Wrong Predictions'] + grouped['Correct Predictions']
grouped['Accuracy'] = grouped['Correct Predictions'] / grouped['Total'] * 100

fig = go.Figure()
fig.update_traces(textposition='outside')
fig.add_trace(go.Bar(
    x=grouped.index,
    y=grouped['Correct Predictions'],
    name='Correct Predictions',
    marker_color='blue'
))

fig.add_trace(go.Bar(
    x=grouped.index,
    y=grouped['Wrong Predictions'],
    name='Wrong Predictions',
    marker_color='red',
    base=grouped['Correct Predictions']
))




fig.update_layout(
    title='Correct vs Wrong Predictions by Data Type',
    xaxis_title='Data Type',
    yaxis_title='Number of Predictions',
    barmode='stack',
    width=800,
    height=800  
)



for i, (type_name, accuracy) in enumerate(zip(grouped.index, grouped['Accuracy'])):
    fig.add_annotation(
        x=type_name,
        y=grouped['Total'].iloc[i] + 10,
        text=f'Acc: {accuracy:.1f}%',
        showarrow=False,
        font=dict(size=12, color='black'),
        align='center',
    )
fig.show()